## Importing required libraries

In [ ]:
import numpy as np
import pandas as pd
import re

## Importing CSV file

In [ ]:
dataDF = pd.read_csv("customer_data.csv",index_col="customer_id")
dataDF

In [ ]:
# information about columns
dataDF.info()

In [ ]:
# checking sample
dataDF.sample(8)

## Cleaning data

In [ ]:
dataDF.shape    # (1000,7)
dataDF.value_counts()

length shown is 982 and not 1000
therefore null rows present

### 1. Null values

In [ ]:
# dataDF[dataDF.isnull()==True]
null_count = dataDF.isnull().sum()
null_count

3 columns have 18 NaN each

#### Removing null values

In [ ]:
# cannot drop whole row for NaN mobile and email, might affect data analysis
# mobile number and email is not mandatory for statistics, therefore zero value will not affect analysis

dataDF.loc[:,'mobile_number'].isnull().sum() # before filling = 18
dataDF.mobile_number = dataDF.loc[:,'mobile_number'].fillna(0) # filling
dataDF.loc[:,'mobile_number'].isnull().sum() # after filling

In [ ]:
# cannot drop whole row for NaN mobile and email, might affect data analysis
# mobile number and email is not mandatory for statistics, therefore zero value will not affect analysis

dataDF.loc[:,'email'].isnull().sum() # before filling = 18
dataDF.email = dataDF.loc[:,'email'].fillna("") # filling
dataDF.loc[:,'email'].isnull().sum() # after filling

In [ ]:
# gender might be required for statistics
# no way to predict gender based on other columns or rows
# better to drop

dataDF.loc[:,'gender'].isnull().sum() # before dropping = 18
dataDF = dataDF[dataDF['gender'].notna()]
dataDF.loc[:,'gender'].isnull().sum() # after dropping

#### Email ID

### 2. Formatting

#### Email ID

In [ ]:
regex_expression_email = re.compile(r'^[a-zA-Z0-9_.]+@[a-zA-Z0-9\-]+\.[a-zA-Z_.]+$')

bool_values = dataDF.email.apply(lambda x: bool(regex_expression_email.match(x)))

bool_values = pd.concat([bool_values, dataDF.email], axis=1)
bool_values.columns=['bool_value','email']
bool_values.loc[bool_values.bool_value==False]

bool_values is empty => all emails are in appropriate format

#### Mobile Number

In [ ]:
dataDF.mobile_number.isnull().unique()
# no null entries

abc = dataDF.mobile_number.str.split('-')
abc.str.join('')
abc
# now no more dashes in mobile numbers